# QSRR Model 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Descriptors imported from RDKit (n=208)
all_descriptor_names = [desc[0] for desc in Descriptors._descList]   
calc = MoleculeDescriptors.MolecularDescriptorCalculator(all_descriptor_names)

# Functions that calculate descriptors from SMILES
def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [np.nan] * len(all_descriptor_names)
    return calc.CalcDescriptors(mol)

# Check the validity of the SMILES
def is_valid_molecule(smiles):
    return Chem.MolFromSmiles(smiles) is not None

In [3]:
# Loading data
df = pd.read_csv("DTB.csv") 
df = df.dropna(subset=["SMILES", "RI"])  
df = df[df["SMILES"].apply(is_valid_molecule)].copy() 

# Descriptors
df["Descriptors"] = df["SMILES"].apply(compute_descriptors)
desc_df = pd.DataFrame(df["Descriptors"].tolist(), columns=all_descriptor_names)
df = pd.concat([df.reset_index(drop=True), desc_df], axis=1)
df.drop(columns=["Descriptors"], inplace=True)

print(f"\nGeneral statistics:")
print(f"Total molecules in the database: {df.shape[0]} molecules")


General statistics:
Total molecules in the database: 166 molecules


In [5]:
def prediction(smiles_list, dtb_df, descriptor_names=None, n_iterations=500, k=140, model_type='Ridge'):
    # Calculation of descriptors
    inc_descs = [compute_descriptors(smi) for smi in smiles_list]
    if descriptor_names is None:
        descriptor_names = [col for col in all_descriptor_names if col in dtb_df.columns]
    inc_df = pd.DataFrame(inc_descs, columns=descriptor_names)

    # Check that the columns exist
    missing_cols = [col for col in descriptor_names if col not in dtb_df.columns]
    if missing_cols:
        raise ValueError(f"Columns missing from the database : {missing_cols}")

    # Filtering valid data
    subset = dtb_df.dropna(subset=descriptor_names + ['RI']).copy()

    # Cleaning problematic columns
    X_df = subset[descriptor_names].replace([np.inf, -np.inf], np.nan)
    X_df = X_df.dropna(axis=1)  
    X_df = X_df.loc[:, X_df.std() > 0]  
    descriptor_names = X_df.columns.tolist()
    X = X_df.values
    y = subset["RI"].values

    # Standardization
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    inc_scaled = scaler.transform(inc_df[descriptor_names])  

    # Selection of the best descriptors
    selector = SelectKBest(score_func=f_regression, k=min(k, X_scaled.shape[1]))
    X_selected = selector.fit_transform(X_scaled, y)
    selected_mask = selector.get_support()
    inc_selected = inc_scaled[:, selected_mask]

    predictions_all = []
    r2_cv_model = []
    rel_error_model = []

    for _ in range(n_iterations):
        X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, shuffle=True)

        if model_type == 'Linear':
            model = LinearRegression()
        elif model_type == 'Ridge':
            model = Ridge(alpha=1.0)
        elif model_type == 'PLS':
            model = PLSRegression(n_components=min(4, k))
        else:
            raise ValueError("Model not supported : " + model_type)

        model.fit(X_train, y_train)

        # Test prediction (20%) 
        y_pred = model.predict(X_test)
        relative_error = np.mean(np.abs((y_test - y_pred) / y_pred)) * 100
        rel_error_model.append(relative_error)

        # Cross-validation
        kf = KFold(n_splits=6, shuffle=True, random_state=42)
        r2_cv = cross_val_score(model, X_selected, y, cv=kf, scoring='r2')
        r2_cv_model.append(r2_cv.mean())

        # Prediction on unknown molecules
        preds = model.predict(inc_selected)
        predictions_all.append(preds)

    predictions_all = np.array(predictions_all)
    mean_preds = predictions_all.mean(axis=0)
    std_preds = predictions_all.std(axis=0)

    # Model summary
    print(f"Model used : {model_type}")
    print(f"Selected descriptors : {int(sum(selected_mask))}")
    print(f"R² cross-validation (medium): {round(np.mean(r2_cv_model), 3)}")
    print(f"Average relative error (%) on test (20%) : {round(np.mean(rel_error_model), 2)}\n")

    # Prediction results
    results = []
    for i, smi in enumerate(smiles_list):
        results.append({
            "SMILES": smi,
            "RI predicts (moy)": round(mean_preds[i], 2),
           "Standard deviation": round(std_preds[i], 2)
        })

    return pd.DataFrame(results)

In [6]:
# Prediction
smiles_inconnus = ['CCCCCCCCC']
resultats = prediction(smiles_inconnus, df, n_iterations=300, k=90, model_type="Ridge")
print(resultats)

Model used : Ridge
Selected descriptors : 90
R² cross-validation (medium): 0.982
Average relative error (%) on test (20%) : 2.75

      SMILES  RI predicts (moy)  Standard deviation
0  CCCCCCCCC              925.2                8.01
